In [ ]:
%load_ext autoreload
%autoreload
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from DarcySolver import solve_darcy
import sympy as sym

T = 2           # final time
num_steps = 10    # number of time steps
dt = T / num_steps
resolutions = [5,10,20,40]

K_value = 1e-10
boundary_all = CompiledSubDomain("on_boundary")
boundary_right = CompiledSubDomain("on_boundary && near(x[0], 1)")
boundary_left = CompiledSubDomain("on_boundary && near(x[0], 0)")
boundary_top = CompiledSubDomain("on_boundary && near(x[1], 1)")
boundary_bottom = CompiledSubDomain("on_boundary && near(x[1], 0)")
x, y = sym.symbols('x[0], x[1]')
t, K = sym.symbols('t, K')

#p_e = (1 + x**2 + 2*y**2)
p_e = (sym.sin(2*x)*sym.sin(5*y)*4)*t
#p_e = sym.exp(1+t**2 + x**2 + y**2)

f = - K* (sym.diff(sym.diff(p_e, x), x) + sym.diff( sym.diff(p_e, y), y) ) + sym.diff(p_e, t)
f = sym.simplify(f)
p_N_right = - sym.diff( p_e, x).subs(x, 1)
p_N_left = sym.diff(p_e, x).subs(x, 0)
p_N_bottom = sym.diff(p_e, y).subs(y, 0)
p_N_top = - sym.diff(p_e, y).subs(y, 1)

variables = [p_e, f, p_N_right, p_N_left, p_N_bottom, p_N_top]
variables_code = [sym.printing.ccode(var) for var in variables]
variables = [Expression(var,t=0,K=K_value, degree=3) for var in variables_code]
p_e, f, p_N_right, p_N_left, p_N_bottom, p_N_top = variables

def compute_order(error, h):
    h = np.array(h)
    err_ratio = np.array(error[:-1]) / np.array(error[1:])
    return np.log(err_ratio)/np.log(h[:-1] / h[1:])

In [ ]:
def darcy_convergence(resolutions, boundaries, boundary_conditions):
    L2_errors = []
    h = []
    for N in resolutions:
        p_e.t = 0
        mesh = UnitSquareMesh(N, N)
        h.append(mesh.hmax())
        boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
        
        for b, marker_id in boundaries.items():
            b.mark(boundary_marker, marker_id)

        solution = solve_darcy(mesh, f, T, num_steps, K_value,
                               boundary_marker, boundary_conditions,
                               p_initial=p_e, degree=1, theta=0.5)
        solution = [s.copy() for s in solution]
        J = 0
        for i,p in enumerate(solution):
            p_e.t = (i +1)*dt
            J += errornorm(p_e, p, "L2")
        L2_errors.append(J/num_steps)
    return L2_errors, h, solution

In [ ]:
# Dirichlet boundary conditions
boundaries = {boundary_all:1}
boundary_conditions = {1:{"Dirichlet":p_e}}
L2_errors, h, solution = darcy_convergence(resolutions, boundaries, boundary_conditions)
compute_order(L2_errors, h)

In [ ]:
# Neumann boundary conditions
boundaries = {boundary_top: 1,
              boundary_bottom: 1,
              boundary_left: 2,
              boundary_right: 3}

boundary_conditions = {1:{"Dirichlet": p_e},
                       2:{"Neumann": p_N_left},
                       3:{"Neumann": p_N_right},
                       }
L2_errors, h, solution = darcy_convergence(resolutions, boundaries, boundary_conditions)
compute_order(L2_errors, h)

In [ ]:
# Robin boundary conditions
boundaries = {boundary_top: 1,
              boundary_bottom: 1,
              boundary_left: 2,
              boundary_right: 3}
beta = 0.5
boundary_conditions = {1:{"Dirichlet":p_e},
                       2:{"Robin": (beta, beta*p_e + p_N_left) },
                       3:{"Robin": (beta, beta*p_e + p_N_right) },
                       }

L2_errors, h, solution = darcy_convergence(resolutions, boundaries, boundary_conditions)
compute_order(L2_errors, h)